In [1]:
# SQLAlchemy
from sqlalchemy import create_engine
import pandas as pd

# Path to sqlite
database_path = "corgies.db"

In [2]:
# Create an engine that can talk to the database
engine = create_engine(f"sqlite:///{database_path}")

# Read Records with Select 

In [3]:
# Query All Records in the the Database
data = engine.execute("SELECT * FROM pets")

for record in data:
    print(record)
    # note how you can use . notation to extract specific columns
    print(record.name)

(1, 'Patterson', 11, 'Corgi', '8 weeks')
Patterson
(2, 'Judson', 10, 'Corgi', '3 years')
Judson


In [4]:
# get all the records as a list:
data = engine.execute("SELECT * FROM pets").fetchall()
data

[(1, 'Patterson', 11, 'Corgi', '8 weeks'),
 (2, 'Judson', 10, 'Corgi', '3 years')]

In [5]:
# get just the first one:
data = engine.execute("SELECT * FROM pets").first()
data

(1, 'Patterson', 11, 'Corgi', '8 weeks')

In [6]:
# what happens when I want just a single value?
data = engine.execute("select age from pets where name = 'Patterson'").first()
data

(11,)

In [7]:
# to actually get the value I have to pull it out of the tuple
data[0]

11

#### Is there a better way to just get one value?

In [8]:
# get a single value ( no extracting from a tuple)
data = engine.execute("select age from pets where name = 'Patterson'").scalar()
data

11

## Read into Pandas DataFrame

In [9]:
# we need an explicit connection ( execute creates one behind the scenes)
conn = engine.connect()

In [10]:
pets = pd.read_sql("select * from pets", conn)
pets

,id,name,age,breed,age_adopted
0,1,Patterson,11,Corgi,8 weeks
1,2,Judson,10,Corgi,3 years


In [11]:
training = pd.read_sql("select * from training", conn)
training

,id,task,description
0,1,sit,sit with bottom on floor
1,2,I feel faint,"lay flat on back legs up like a ""faint"""
2,3,roll over,completely roll over
3,4,speak,bark loud and clear
4,5,lay down,lay with all four legs relaxed


In [12]:
grades = pd.read_sql("select * from grades", conn)
grades

,pet_id,task_id,grade,comments
0,1,1,A,good
1,1,2,A-,"quick but needed ""do a good one"""
2,1,3,B,weak attempt
3,1,4,B,quiet
4,1,5,C,reluctant
5,2,1,B,required butt bump
6,2,2,B-,more of a swoon
7,2,4,A,loud and deep
8,2,5,A,good


### Now let's go ahead and merge the DataFrames

In [13]:
merged_data = grades.merge(training, left_on="task_id", right_on="id").\
    merge(pets, left_on="pet_id", right_on="id")
merged_data

,pet_id,task_id,grade,comments,id_x,task,description,id_y,name,age,breed,age_adopted
0,1,1,A,good,1,sit,sit with bottom on floor,1,Patterson,11,Corgi,8 weeks
1,1,2,A-,"quick but needed ""do a good one""",2,I feel faint,"lay flat on back legs up like a ""faint""",1,Patterson,11,Corgi,8 weeks
2,1,3,B,weak attempt,3,roll over,completely roll over,1,Patterson,11,Corgi,8 weeks
3,1,4,B,quiet,4,speak,bark loud and clear,1,Patterson,11,Corgi,8 weeks
4,1,5,C,reluctant,5,lay down,lay with all four legs relaxed,1,Patterson,11,Corgi,8 weeks
5,2,1,B,required butt bump,1,sit,sit with bottom on floor,2,Judson,10,Corgi,3 years
6,2,2,B-,more of a swoon,2,I feel faint,"lay flat on back legs up like a ""faint""",2,Judson,10,Corgi,3 years
7,2,4,A,loud and deep,4,speak,bark loud and clear,2,Judson,10,Corgi,3 years
8,2,5,A,good,5,lay down,lay with all four legs relaxed,2,Judson,10,Corgi,3 years


In [14]:
merged_data.columns

Index(['pet_id', 'task_id', 'grade', 'comments', 'id_x', 'task', 'description',
       'id_y', 'name', 'age', 'breed', 'age_adopted'],
      dtype='object')

In [15]:
# drop columns
merged_data = merged_data.drop(['pet_id', 'task_id', 'id_x', 'description',
       'id_y', 'age', 'breed', 'age_adopted'], axis=1)
merged_data

,grade,comments,task,name
0,A,good,sit,Patterson
1,A-,"quick but needed ""do a good one""",I feel faint,Patterson
2,B,weak attempt,roll over,Patterson
3,B,quiet,speak,Patterson
4,C,reluctant,lay down,Patterson
5,B,required butt bump,sit,Judson
6,B-,more of a swoon,I feel faint,Judson
7,A,loud and deep,speak,Judson
8,A,good,lay down,Judson


In [16]:
merged_data.columns

Index(['grade', 'comments', 'task', 'name'], dtype='object')

In [17]:
# reorder columns
merged_data = merged_data[['name',  'task', 'grade', 'comments']]
merged_data


,name,task,grade,comments
0,Patterson,sit,A,good
1,Patterson,I feel faint,A-,"quick but needed ""do a good one"""
2,Patterson,roll over,B,weak attempt
3,Patterson,speak,B,quiet
4,Patterson,lay down,C,reluctant
5,Judson,sit,B,required butt bump
6,Judson,I feel faint,B-,more of a swoon
7,Judson,speak,A,loud and deep
8,Judson,lay down,A,good


#### Let's move all that work to the database

In [18]:
sql = """
select name, task, grade, comments 
from pets p
join grades g on g.pet_id = p.id
join training t on t.id = g.task_id
order by task_id
"""

training_df = pd.read_sql(sql, conn)
training_df

,name,task,grade,comments
0,Patterson,sit,A,good
1,Judson,sit,B,required butt bump
2,Patterson,I feel faint,A-,"quick but needed ""do a good one"""
3,Judson,I feel faint,B-,more of a swoon
4,Patterson,roll over,B,weak attempt
5,Patterson,speak,B,quiet
6,Judson,speak,A,loud and deep
7,Patterson,lay down,C,reluctant
8,Judson,lay down,A,good


#### If we're going to do this often a view would be nice

In [19]:
sql = "select * from pet_training where name = 'Patterson' order by grade"
training_view_df = pd.read_sql(sql, conn)
training_view_df

,name,task,grade,comments
0,Patterson,sit,A,good
1,Patterson,I feel faint,A-,"quick but needed ""do a good one"""
2,Patterson,roll over,B,weak attempt
3,Patterson,speak,B,quiet
4,Patterson,lay down,C,reluctant


## Panada "insert"

The to_sql method can create, replace and append to tables.  Very quick and easy!

In [20]:
training_view_df.to_sql(name="patterson_grade", index_label='id', if_exists='replace', con=conn)

In [25]:
# did the table get created?
engine.execute("select * from patterson_grade order by id").fetchall()

[(0, 'Patterson', 'sit', 'A', 'good'),
 (1, 'Patterson', 'I feel faint', 'A-', 'quick but needed "do a good one"'),
 (2, 'Patterson', 'roll over', 'B', 'weak attempt'),
 (3, 'Patterson', 'speak', 'B', 'quiet'),
 (4, 'Patterson', 'lay down', 'C', 'reluctant')]

## Create with Insert

In [21]:
sql = """
insert into pets (id, name, age, breed, age_adopted)
values ( 3, 'Cool Corgi', 4, 'Cardigan', 2)
"""
engine.execute(sql)

pd.read_sql("select * from pets", conn)

,id,name,age,breed,age_adopted
0,1,Patterson,11,Corgi,8 weeks
1,2,Judson,10,Corgi,3 years
2,3,Cool Corgi,4,Cardigan,2


## Update with Update

In [22]:
sql = "update pets set age_adopted = '2 Years' where id = 3"
engine.execute(sql)

pd.read_sql("select * from pets", conn)

,id,name,age,breed,age_adopted
0,1,Patterson,11,Corgi,8 weeks
1,2,Judson,10,Corgi,3 years
2,3,Cool Corgi,4,Cardigan,2 Years


## Delete with Delete

In [23]:
sql = "delete from pets where id = 3"
engine.execute(sql)

pd.read_sql("select * from pets", conn)

,id,name,age,breed,age_adopted
0,1,Patterson,11,Corgi,8 weeks
1,2,Judson,10,Corgi,3 years


In [24]:
conn.close()